In [5]:
from models.model import Net
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

model = Net(num_block=3, num_class=50, num_landmark=390, max_length=124, embed_dim=512, num_head=16,in_channels=390,kernel_size=17)
checkpoint = torch.load('output/weights/cfg_3/_42/checkpoint_best_seed-1.pth', map_location=torch.device('cpu'))
model_state_dict = checkpoint['model']
model.load_state_dict(model_state_dict)
import torch
import torchvision.models as models
input_data = torch.randn(1,124, 390)
model(input_data)
# Chuyển model sang ONNX
torch.onnx.export(model,                      # model
                    input_data,                # dữ liệu đầu vào mẫu
                    "output/model.onnx",              # tên file output
                    export_params=True,        # chuyển cả trọng số của model
                    do_constant_folding=True,  # folding các biến hằng trong model để tối ưu
                    input_names=['input'],     # tên của các đầu vào của model
                    output_names=['output'])   # tên của các đầu ra của model

c:\Users\Admin\anaconda3\Lib\site-packages\torch\onnx\symbolic_helper.py:1513: UserWarning: ONNX export mode is set to TrainingMode.EVAL, but operator 'dropout' is set to train=True. Exporting with train=True.
  warnings.warn(


In [6]:


def interpolate_or_pad(data, max_len=100, mode="start"):
    diff = max_len - data.shape[0]
    if diff <= 0:  # Crop
        data = F.interpolate(data.permute(1,2,0),max_len).permute(2,0,1)
        mask = torch.ones_like(data[:,0,0])
        return data, mask
    coef = 0
    padding = torch.ones((diff, data.shape[1], data.shape[2]))
    mask = torch.ones_like(data[:,0,0])
    data = torch.cat([data, padding * coef])
    mask = torch.cat([mask, padding[:,0,0] * coef])
    return data, mask
class Preprocessing(nn.Module):
    def __init__(self):
        super(Preprocessing, self).__init__()
    def normalize(self,x):
        nonan = x[~torch.isnan(x)].view(-1, x.shape[-1])
        x = x - nonan.mean(0)[None, None, :]
        x = x / nonan.std(0, unbiased=False)[None, None, :]
        return x
    def fill_nans(self,x):
        x[torch.isnan(x)] = 0
        return x
        
    def forward(self, x):
        #seq_len, 3* n_landmarks -> seq_len, n_landmarks, 3
        x = x.reshape(x.shape[0],3,-1).permute(0,2,1)
        # Normalize & fill nans
        x = self.normalize(x)
        x = self.fill_nans(x)
        return x
preprocess = Preprocessing()
x = np.load('./dataset/train_landmarks_npy/VSL50/bucminh_pg_1.npy')
x = torch.from_numpy(x)
x = preprocess(x)
x, mask = interpolate_or_pad(x, max_len=124)
x = x.reshape(x.shape[0],-1)
x = x.unsqueeze(0)
x = x.numpy().astype(np.float32)
x.shape

(1, 124, 390)

In [17]:
import json
import onnxruntime
sess = onnxruntime.InferenceSession("output/model.onnx")
def predict(input_data, session, label_map,threshold=0.4):
    input_data = np.expand_dims(input_data, axis=0).astype(np.float32)
    # Chạy mô hình trên dữ liệu đầu vào
    output = session.run(None, {'input': input_data})[0][0]
    predictions = np.argmax(output, axis=0)
    probabilities = (np.exp(output) / np.sum(np.exp(output), axis=0))
    confidence = probabilities[np.argmax(probabilities, axis=0)]
    predicted_labels = list(label_map.keys())[predictions]
    if confidence < threshold :
        return "Uncertain"
    else:
        return predicted_labels
with open('dataset/sign_to_prediction_index_map.json', 'r', encoding='utf-8') as json_file:
    label_map = json.load(json_file)

ranx = np.random.randn(124,390)
predicted_labels = predict(ranx, sess, label_map,0.7)

print("Predicted labels:", predicted_labels)

Predicted labels: không cho


In [12]:
import cv2
import mediapipe as mp
import numpy as np
import json
import onnxruntime

sess = onnxruntime.InferenceSession("output/model.onnx")
def predict(input_data, session, label_map,threshold=0.5):
    input_data = np.expand_dims(input_data, axis=0).astype(np.float32)
    # Chạy mô hình trên dữ liệu đầu vào
    output = session.run(None, {'input': input_data})[0][0]
    predictions = np.argmax(output, axis=0)
    probabilities = (np.exp(output) / np.sum(np.exp(output), axis=0))
    confidence = probabilities[np.argmax(probabilities, axis=0)]
    predicted_labels = list(label_map.keys())[predictions]
    if confidence < threshold :
        return "Uncertain"
    else:
        return predicted_labels
with open('dataset/sign_to_prediction_index_map.json', 'r', encoding='utf-8') as json_file:
    label_map = json.load(json_file)

NOSE=[
    1,2,98,327
]
LNOSE = [98]
RNOSE = [327]
LIP = [ 0, 
    61, 185, 40, 39, 37, 267, 269, 270, 409,
    291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
    95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
]
LLIP = [84,181,91,146,61,185,40,39,37,87,178,88,95,78,191,80,81,82]
RLIP = [314,405,321,375,291,409,270,269,267,317,402,318,324,308,415,310,311,312]

POSE = [500, 502, 504, 501, 503, 505, 512, 513]
LPOSE = [513,505,503,501]
RPOSE = [512,504,502,500]

LARMS = [501, 503, 505, 507, 509, 511]
RARMS = [500, 502, 504, 506, 508, 510]

REYE = [
    33, 7, 163, 144, 145, 153, 154, 155, 133,
    246, 161, 160, 159, 158, 157, 173,
]
LEYE = [
    263, 249, 390, 373, 374, 380, 381, 382, 362,
    466, 388, 387, 386, 385, 384, 398,
]

LHAND = np.arange(468, 489).tolist()
RHAND = np.arange(522, 543).tolist()

POINT_LANDMARKS = LIP + LHAND + RHAND + NOSE + REYE + LEYE + LARMS + RARMS
def extract_keypoints(results,POINT_LANDMARKS):
    face_x  = np.array([res.x for res in results.face_landmarks.landmark],dtype=np.float32) if results.face_landmarks else np.zeros(468)
    face_y  = np.array([res.y for res in results.face_landmarks.landmark],dtype=np.float32) if results.face_landmarks else np.zeros(468)
    face_z  = np.array([res.z for res in results.face_landmarks.landmark],dtype=np.float32) if results.face_landmarks else np.zeros(468)
    
    lh_x  = np.array([res.x for res in results.left_hand_landmarks.landmark],dtype=np.float32) if results.left_hand_landmarks else np.zeros(21)
    lh_y  = np.array([res.y for res in results.left_hand_landmarks.landmark],dtype=np.float32) if results.left_hand_landmarks else np.zeros(21)
    lh_z  = np.array([res.z for res in results.left_hand_landmarks.landmark],dtype=np.float32) if results.left_hand_landmarks else np.zeros(21)
    
    pose_x  = np.array([res.x for res in results.pose_landmarks.landmark],dtype=np.float32) if results.pose_landmarks else np.zeros(33)
    pose_y  = np.array([res.y for res in results.pose_landmarks.landmark],dtype=np.float32) if results.pose_landmarks else np.zeros(33)
    pose_z  = np.array([res.z for res in results.pose_landmarks.landmark],dtype=np.float32) if results.pose_landmarks else np.zeros(33)
    
    
    rh_x  = np.array([res.x for res in results.right_hand_landmarks.landmark],dtype=np.float32) if results.right_hand_landmarks else np.zeros(21)
    rh_y  = np.array([res.y for res in results.right_hand_landmarks.landmark],dtype=np.float32) if results.right_hand_landmarks else np.zeros(21)
    rh_z  = np.array([res.z for res in results.right_hand_landmarks.landmark],dtype=np.float32) if results.right_hand_landmarks else np.zeros(21)

    x_cor = np.concatenate([face_x, lh_x, pose_x, rh_x])
    y_cor = np.concatenate([face_y, lh_y, pose_y, rh_y])
    z_cor = np.concatenate([face_z, lh_z, pose_z, rh_z])
    POINT_LANDMARKS_array = np.array(POINT_LANDMARKS)
    result = np.concatenate((x_cor[POINT_LANDMARKS_array], y_cor[POINT_LANDMARKS_array], z_cor[POINT_LANDMARKS_array]))
    return   result
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils 
n_frames = 0
# Mở camera
cap = cv2.VideoCapture('./boiroi_tx_2.mp4')
inputs_model = np.zeros((124, 390))
while cap.isOpened():
    # Đọc frame từ camera
    ret, frame = cap.read()
    if not ret:
        print("Không thể đọc từ camera.")
        break
    
    # Chuyển đổi frame sang màu xám để xử lý nhanh hơn
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Dùng Mediapipe để xử lý landmarks trên khuôn mặt và bàn tay
    results = holistic.process(image=frame)
    keypoint = extract_keypoints(results)
    inputs_model = np.vstack((keypoint, inputs_model))
    if inputs_model.shape[0] >= 124:
        inputs_model =  inputs_model[:124]
    predicted_labels = predict(inputs_model, sess, label_map, 0.7)
    print(predicted_labels)
    # Hiển thị frame
    cv2.imshow('MediaPipe Holistic', frame)
    n_frames +=1
    print(n_frames)
    # Thoát khỏi vòng lặp nếu nhấn phím 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Giải phóng các tài nguyên
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [112]:
class Preprocessing(nn.Module):
    def __init__(self):
        super(Preprocessing, self).__init__()

    def normalize(self,x):
        nonan = x[~torch.isnan(x)].view(-1, x.shape[-1])
        x = x - nonan.mean(0)[None, None, :]
        x = x / nonan.std(0, unbiased=False)[None, None, :]
        return x
    
    def fill_nans(self,x):
        x[torch.isnan(x)] = 0
        return x
        
    def forward(self, x):
        x = torch.Tensor(x)
        #seq_len, 3* n_landmarks -> seq_len, n_landmarks, 3
        x = x.reshape(x.shape[0],3,-1).permute(0,2,1)
        
        # Normalize & fill nans
        x = self.normalize(x)
        x = self.fill_nans(x)
        return x
def interpolate_or_pad(data, max_len=124, mode="start"):
    diff = max_len - data.shape[0]

    if diff <= 0:  # Crop
        data = F.interpolate(data.permute(1,2,0),max_len).permute(2,0,1)
        return data
    coef = 0
    padding = torch.ones((diff, data.shape[1], data.shape[2]))
    data = torch.cat([data, padding * coef])
    return data

In [113]:
x = inputs_model[:87]
print(x.shape)
x = interpolate_or_pad(preprocess(x))
x = x.reshape(x.shape[0],-1)
print(x.shape)
predicted_labels = predict(x, sess, label_map, 0.3)
predicted_labels

(87, 390)
torch.Size([124, 390])


'bối rối'

In [83]:
x = np.load('./boiroi_tx_2.npy')
print(x.shape)
x = preprocess(x)
x, _ = interpolate_or_pad(x, max_len=124)
x = x.reshape(x.shape[0],-1)
print(x.shape)
predicted_labels = predict(x, sess, label_map, 0.3)
predicted_labels

(87, 390)
torch.Size([124, 390])


'bối rồi'

In [26]:
inputs_model[:60].shape

(60, 390)

In [15]:
NOSE=[
    1,2,98,327
]
LNOSE = [98]
RNOSE = [327]
LIP = [ 0, 
    61, 185, 40, 39, 37, 267, 269, 270, 409,
    291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
    95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
]
LLIP = [84,181,91,146,61,185,40,39,37,87,178,88,95,78,191,80,81,82]
RLIP = [314,405,321,375,291,409,270,269,267,317,402,318,324,308,415,310,311,312]
POSE = [500, 502, 504, 501, 503, 505, 512, 513]
LPOSE = [513,505,503,501]
RPOSE = [512,504,502,500]
LARMS = [501, 503, 505, 507, 509, 511]
RARMS = [500, 502, 504, 506, 508, 510]

REYE = [
    33, 7, 163, 144, 145, 153, 154, 155, 133,
    246, 161, 160, 159, 158, 157, 173,
]
LEYE = [
    263, 249, 390, 373, 374, 380, 381, 382, 362,
    466, 388, 387, 386, 385, 384, 398,
]

LHAND = np.arange(468, 489).tolist()
RHAND = np.arange(522, 543).tolist()
POINT_LANDMARKS = LIP + LHAND + RHAND + NOSE + REYE + LEYE + LARMS + RARMS 
def extract_keypoints(results, POINT_LANDMARKS):
    face_x  = np.array([res.x for res in results.face_landmarks.landmark],dtype=np.float32) if results.face_landmarks else np.zeros(468)
    face_y  = np.array([res.y for res in results.face_landmarks.landmark],dtype=np.float32) if results.face_landmarks else np.zeros(468)
    face_z  = np.array([res.z for res in results.face_landmarks.landmark],dtype=np.float32) if results.face_landmarks else np.zeros(468)
    
    lh_x  = np.array([res.x for res in results.left_hand_landmarks.landmark],dtype=np.float32) if results.left_hand_landmarks else np.zeros(21)
    lh_y  = np.array([res.y for res in results.left_hand_landmarks.landmark],dtype=np.float32) if results.left_hand_landmarks else np.zeros(21)
    lh_z  = np.array([res.z for res in results.left_hand_landmarks.landmark],dtype=np.float32) if results.left_hand_landmarks else np.zeros(21)
    
    pose_x  = np.array([res.x for res in results.pose_landmarks.landmark],dtype=np.float32) if results.pose_landmarks else np.zeros(33)
    pose_y  = np.array([res.y for res in results.pose_landmarks.landmark],dtype=np.float32) if results.pose_landmarks else np.zeros(33)
    pose_z  = np.array([res.z for res in results.pose_landmarks.landmark],dtype=np.float32) if results.pose_landmarks else np.zeros(33)
    
    
    rh_x  = np.array([res.x for res in results.right_hand_landmarks.landmark],dtype=np.float32) if results.right_hand_landmarks else np.zeros(21)
    rh_y  = np.array([res.y for res in results.right_hand_landmarks.landmark],dtype=np.float32) if results.right_hand_landmarks else np.zeros(21)
    rh_z  = np.array([res.z for res in results.right_hand_landmarks.landmark],dtype=np.float32) if results.right_hand_landmarks else np.zeros(21)

    x_cor = np.concatenate([face_x, lh_x, pose_x, rh_x])
    y_cor = np.concatenate([face_y, lh_y, pose_y, rh_y])
    z_cor = np.concatenate([face_z, lh_z, pose_z, rh_z])
    POINT_LANDMARKS_array = np.array(POINT_LANDMARKS)
    result = np.concatenate((x_cor[POINT_LANDMARKS_array], y_cor[POINT_LANDMARKS_array], z_cor[POINT_LANDMARKS_array]))
    print(result)
    return result

# POINT_LANDMARKS_idx =np.concatenate((np.array(POINT_LANDMARKS), np.array(POINT_LANDMARKS)+543, np.array(POINT_LANDMARKS)+543*2))
extract_keypoints(results,POINT_LANDMARKS).shape

NameError: name 'results' is not defined

In [16]:
inputs_model = np.zeros((124, 390))
for i in range(150):
    keypoint = extract_keypoints(results)
    inputs_model = np.vstack((keypoint, inputs_model))
    if inputs_model.shape[0] >= 124:
        inputs_model =  inputs_model[:124]
    predicted_labels = predict(inputs_model, sess, label_map, 0.7)
    print(predicted_labels)

NameError: name 'results' is not defined

In [7]:
import cv2
import mediapipe as mp
import numpy as np
import json
import onnxruntime
from data.post_process import Preprocessing, interpolate_or_pad
import sys
sys.path.insert(0, 'F:\6.Spring_24\VIetNamese_sign_language')
from dataset.extract_landmark import POINT_LANDMARKS
import pandas as pd
import json
preprocessLayer = Preprocessing()
sess = onnxruntime.InferenceSession("output/model.onnx")
with open('dataset/sign_to_prediction_index_map.json', 'r', encoding='utf-8') as json_file:
    label_map = json.load(json_file)
def predict(input_data, session, label_map=label_map,threshold=0.5):
    input_data = np.expand_dims(input_data, axis=0).astype(np.float32)
    # Chạy mô hình trên dữ liệu đầu vào
    output = session.run(None, {'input': input_data})[0][0]
    predictions = np.argmax(output, axis=0)
    probabilities = (np.exp(output) / np.sum(np.exp(output), axis=0))
    confidence = probabilities[np.argmax(probabilities, axis=0)]
    predicted_labels = list(label_map.keys())[predictions]
    if confidence < threshold :
        return "Uncertain"
    else:
        return predicted_labels
def extract_keypoints(results,POINT_LANDMARKS):
    face_x  = np.array([res.x for res in results.face_landmarks.landmark],dtype=np.float32) if results.face_landmarks else np.zeros(468)
    face_y  = np.array([res.y for res in results.face_landmarks.landmark],dtype=np.float32) if results.face_landmarks else np.zeros(468)
    face_z  = np.array([res.z for res in results.face_landmarks.landmark],dtype=np.float32) if results.face_landmarks else np.zeros(468)
    
    lh_x  = np.array([res.x for res in results.left_hand_landmarks.landmark],dtype=np.float32) if results.left_hand_landmarks else np.zeros(21)
    lh_y  = np.array([res.y for res in results.left_hand_landmarks.landmark],dtype=np.float32) if results.left_hand_landmarks else np.zeros(21)
    lh_z  = np.array([res.z for res in results.left_hand_landmarks.landmark],dtype=np.float32) if results.left_hand_landmarks else np.zeros(21)
    
    pose_x  = np.array([res.x for res in results.pose_landmarks.landmark],dtype=np.float32) if results.pose_landmarks else np.zeros(33)
    pose_y  = np.array([res.y for res in results.pose_landmarks.landmark],dtype=np.float32) if results.pose_landmarks else np.zeros(33)
    pose_z  = np.array([res.z for res in results.pose_landmarks.landmark],dtype=np.float32) if results.pose_landmarks else np.zeros(33)
    
    
    rh_x  = np.array([res.x for res in results.right_hand_landmarks.landmark],dtype=np.float32) if results.right_hand_landmarks else np.zeros(21)
    rh_y  = np.array([res.y for res in results.right_hand_landmarks.landmark],dtype=np.float32) if results.right_hand_landmarks else np.zeros(21)
    rh_z  = np.array([res.z for res in results.right_hand_landmarks.landmark],dtype=np.float32) if results.right_hand_landmarks else np.zeros(21)

    x_cor = np.concatenate([face_x, lh_x, pose_x, rh_x])
    y_cor = np.concatenate([face_y, lh_y, pose_y, rh_y])
    z_cor = np.concatenate([face_z, lh_z, pose_z, rh_z])
    POINT_LANDMARKS_array = np.array(POINT_LANDMARKS)
    result = np.concatenate((x_cor[POINT_LANDMARKS_array], y_cor[POINT_LANDMARKS_array], z_cor[POINT_LANDMARKS_array]))
    return   result
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
preprocess = Preprocessing()
# Mở camera
cap = cv2.VideoCapture('./boiroi_tx_2.mp4')
sequence=[]
n_frame = 0
num_frame_space = 15
predicted = ''
while cap.isOpened():
    # Đọc frame từ camera
    ret, frame = cap.read()
    if not ret:
        print("Không thể đọc từ camera.")
        break
    
    # Chuyển đổi frame sang màu xám để xử lý nhanh hơn
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Dùng Mediapipe để xử lý landmarks trên khuôn mặt và bàn tay
    results = holistic.process(image=frame)
    keypoints = extract_keypoints(results,POINT_LANDMARKS)
    sequence.append(keypoints)
    sequence_arr = np.array(sequence)
    print(sequence_arr.shape)
sequence_arr = interpolate_or_pad(preprocess(sequence_arr))
sequence_arr = sequence_arr.reshape(sequence_arr.shape[0],-1)
predicted = predict(sequence_arr,sess)
print(predicted)
cap.release()
cv2.destroyAllWindows()


(1, 390)
(2, 390)
(3, 390)
(4, 390)
(5, 390)
(6, 390)
(7, 390)
(8, 390)
(9, 390)
(10, 390)
(11, 390)
(12, 390)
(13, 390)
(14, 390)
(15, 390)
(16, 390)
(17, 390)
(18, 390)
(19, 390)
(20, 390)
(21, 390)
(22, 390)
(23, 390)
(24, 390)
(25, 390)
(26, 390)
(27, 390)
(28, 390)
(29, 390)
(30, 390)
(31, 390)
(32, 390)
(33, 390)
(34, 390)
(35, 390)
(36, 390)
(37, 390)
(38, 390)
(39, 390)
(40, 390)
(41, 390)
(42, 390)
(43, 390)
(44, 390)
(45, 390)
(46, 390)
(47, 390)
(48, 390)
(49, 390)
(50, 390)
(51, 390)
(52, 390)
(53, 390)
(54, 390)
(55, 390)
(56, 390)
(57, 390)
(58, 390)
(59, 390)
(60, 390)
(61, 390)
(62, 390)
(63, 390)
(64, 390)
(65, 390)
(66, 390)
(67, 390)
(68, 390)
(69, 390)
(70, 390)
(71, 390)
(72, 390)
(73, 390)
(74, 390)
(75, 390)
(76, 390)
(77, 390)
(78, 390)
(79, 390)
(80, 390)
(81, 390)
(82, 390)
(83, 390)
(84, 390)
(85, 390)
(86, 390)
(87, 390)
Không thể đọc từ camera.
bối rối
